In [1]:
from matplotlib import pyplot as plt

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

In [3]:
from matplotlib.patches import FancyArrowPatch

In [4]:
%matplotlib widget
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')

In [5]:
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [6]:
import allel
import numpy as np
import pandas as pd
import seaborn as sns
import umap

## PCA using scikit-learn

http://alimanfoo.github.io/2015/09/28/fast-pca.html

The data:

In [7]:
vcf = allel.read_vcf('../steps/recode_vcf/chr7_females.recode.vcf')

In [8]:
pop = pd.read_table('../data/females.txt', delimiter = '\t', header = None,
                   names = ['Individual', 'Population'])

In [9]:
gt = allel.GenotypeArray(vcf['calldata/GT'])
gt

<GenotypeArray shape=(5497845, 99, 2) dtype=int8>
0/1 0/1 0/1 0/1 0/1 ... 0/1 0/1 0/1 0/1 0/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/1 0/1 0/1 0/1 0/1 ... 0/0 0/0 0/1 0/0 0/1
...
./. 0/0 0/0 ./. ./. ... ./. ./. ./. ./. ./.
./. ./. 0/0 ./. ./. ... 0/0 0/0 0/0 0/0 ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.

In [10]:
ac = gt.count_alleles()
ac

<AlleleCountsArray shape=(5497845, 5) dtype=int32>
105  93   0   0   0
198   0   0   0   0
157  40   1   0   0
...
132   0   0   0   0
173   1   0   0   0
  0   0   0   0   0

Removing singletons and multiallelic SNPs.

In [11]:
flt = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 1)
gf = gt.compress(flt, axis=0)
gf

<GenotypeArray shape=(2949631, 99, 2) dtype=int8>
0/1 0/1 0/1 0/1 0/1 ... 0/1 0/1 0/1 0/1 0/1
0/0 0/0 1/1 1/1 0/1 ... 0/1 0/1 0/1 0/1 0/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
./. ./. 0/0 ./. 0/0 ... 0/0 0/0 ./. ./. ./.
./. ./. 0/0 ./. 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 ./. ... ./. 1/1 ./. ./. ./.

The final input for sklearnPCA:

In [12]:
gn = gf.to_n_alt()
gn

array([[1, 1, 1, ..., 1, 1, 1],
       [0, 0, 2, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [ ]:
sklearn_pca = sklearnPCA(n_components=2)
sklearn_transf = sklearn_pca.fit_transform(gn.T)

In [ ]:
pc1_var = sklearn_pca.explained_variance_ratio_[0].round(decimals=3)*100
pc2_var = sklearn_pca.explained_variance_ratio_[1].round(decimals=3)*100

Creating a dataframe from the resulting array and adding population names to each individual

In [ ]:
principal_df = pd.DataFrame(data = sklearn_transf, 
                           columns = ['PC1', 'PC2'])

In [ ]:
final_df = pd.concat([principal_df, pop[['Population']]], axis = 1)
final_df

In [ ]:
fig, ax = plt.subplots(figsize = (6,5))

groups = final_df.groupby('Population')
for name, group in groups:
    ax.scatter(group['PC1'], group['PC2'], label=name)

ax.set_xlabel('Principal Component 1 ({}%)'.format(pc1_var), fontsize = 10)
ax.set_ylabel('Principal Component 2 ({}%)'.format(pc2_var), fontsize = 10)
ax.set_title('Chromosome 7: Females', fontsize = 15)
ax.legend()
plt.show()

### Effect of pruning

In [ ]:
def plot_ld(gn, title):
    m = allel.rogers_huff_r(gn) ** 2
    ax = allel.plot_pairwise_ld(m)
    ax.set_title(title)
    ax.figure.set_size_inches(5, 5)
    plt.show()

In [ ]:
plot_ld(gn[:1000], 'Figure 1. Pairwise LD.')

In [ ]:
def ld_prune(gn, size, step, threshold=.1, n_iter=1):
    for i in range(n_iter):
        loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gn = gn.compress(loc_unlinked, axis=0)
    return gn

In [ ]:
gnu = ld_prune(gn, size=500, step=200, threshold=.1, n_iter=5)

In [ ]:
plot_ld(gnu[:1000], 'Figure 2. Pairwise LD after LD pruning.')

In [ ]:
# Performing the PCA analysis again
sklearn_pca_pruned = sklearnPCA(n_components=2)
pruned_pca = sklearn_pca_pruned.fit_transform(gnu.T)

# Creating a dataframe and adding population names
principal_pruned = pd.DataFrame(data = pruned_pca, 
                           columns = ['PC1', 'PC2'])
final_pruned = pd.concat([principal_pruned, pop[['Population']]], axis = 1)

# Percentage variance explained
pruned_pc1_var = sklearn_pca_pruned.explained_variance_ratio_[0].round(decimals=3)*100
pruned_pc2_var = sklearn_pca_pruned.explained_variance_ratio_[1].round(decimals=3)*100

In [ ]:
fig, ax = plt.subplots(figsize = (6,5))

groups = final_pruned.groupby('Population')
for name, group in groups:
    ax.scatter(group['PC1'], group['PC2'], label=name)

ax.set_xlabel('Principal Component 1 ({}%)'.format(pruned_pc1_var), fontsize = 10)
ax.set_ylabel('Principal Component 2 ({}%)'.format(pruned_pc2_var), fontsize = 10)
ax.set_title('Pruned chromosome 7: Females', fontsize = 15)
ax.legend()
plt.show()

### Effect of scaling

Performing the PCA analysis with scaling

In [ ]:
gn.shape

In [ ]:
pipeline = make_pipeline(StandardScaler(), sklearnPCA(n_components=2))
scaled_data_transf = pipeline.fit_transform(gn[:1800000, :].T)

Creating a dataframe and adding population names

In [ ]:
principal_scaled = pd.DataFrame(data = scaled_data_transf, 
                           columns = ['PC1', 'PC2'])
final_scaled = pd.concat([principal_scaled, pop[['Population']]], axis = 1)

Percentage variance explained

In [ ]:
pca = pipeline.get_params()['pca']

In [ ]:
scaled_pc1_var = pca.explained_variance_ratio_[0].round(decimals=3)*100
scaled_pc2_var = pca.explained_variance_ratio_[1].round(decimals=3)*100

Plotting the result

In [ ]:
fig, ax = plt.subplots(figsize = (6,5))

groups = final_scaled.groupby('Population')
for name, group in groups:
    ax.scatter(group['PC1'], group['PC2'], label=name)

ax.set_xlabel('Principal Component 1 ({}%)'.format(scaled_pc1_var), fontsize = 10)
ax.set_ylabel('Principal Component 2 ({}%)'.format(scaled_pc2_var), fontsize = 10)
ax.set_title('Scaled chromosome 7: Females', fontsize = 15)
ax.legend()
plt.show()

### Applying both pruning and scaling

In [ ]:
scaled_pruned = pipeline.fit_transform(gnu.T)

In [ ]:
principal_scaled_pruned = pd.DataFrame(data = scaled_pruned, 
                           columns = ['PC1', 'PC2'])
final_scaled_pruned = pd.concat([principal_scaled_pruned, pop[['Population']]], axis = 1)

In [ ]:
pca = pipeline.get_params()['pca']
scaled_pruned_pc1_var = pca.explained_variance_ratio_[0].round(decimals=3)*100
scaled_pruned_pc2_var = pca.explained_variance_ratio_[1].round(decimals=3)*100

In [ ]:
fig, ax = plt.subplots(figsize = (6,5))

groups = final_scaled_pruned.groupby('Population')
for name, group in groups:
    ax.scatter(group['PC1'], group['PC2'], label=name)

ax.set_xlabel('Principal Component 1 ({}%)'.format(scaled_pruned_pc1_var), fontsize = 10)
ax.set_ylabel('Principal Component 2 ({}%)'.format(scaled_pruned_pc2_var), fontsize = 10)
ax.set_title('Scaled & pruned chromosome 7: Females', fontsize = 15)
ax.legend()
plt.show()